# CNN Model For Image Classification
***********************************

## The Model
The model has a simple architecture, similar to LeNet, owing to the similarity of the task to be performed.

We maintain two lists, train_x and train_y. 

We fetch the names of the files from the 'training_labels.csv' file. For each row in the csv file, we fetch the name of the file and the label associated with it. 

The image file is fetched using its name from the input folder.

Since the sizes of the images in the training folder are not same, we resize them to a common dimension of 600x1000

This image is then converted to a numpy array, normalised by dividing it by 255, and its dimensions are expanded by adding a third dimension, the number of color channels, which is set to 1.

This array is then appended to the train_x list.

We one hot encode the label manually. A temporary list with 16 0s is initialised and the index of the list which is equal to the label is made 1. This array is appended to test_y.

Finally the lists are converted to numpy array and returned from the load data function.

For the testing set, the images were taken from the folder and then preprocesses in the same way as the training instances were. The predicted class was stored in a list and later converted to numoy array and then to a dataframe.



In [1]:
import math, re, os
import tensorflow as tf
from tensorflow import keras
import numpy as np 
import pandas as pd 
from PIL import Image
import os
from tensorflow.keras.layers import Conv2D, MaxPooling2D, AveragePooling2D, Flatten, Dense
from tensorflow.keras.models import Sequential
from skimage.io import imread
from skimage.transform import resize

!nvidia-smi

def batch_generator(Train_df, batch_size, steps):
    idx=1
    while True: 
        yield load_data(Train_df,idx-1,batch_size)
        if idx<steps:
            idx+=1
        else:
            idx=1

def load_data(Train_df, idx, batch_size):
    dirname = '/kaggle/input/datathonindoml-2022/train/train/'
    df = pd.read_csv(Train_df, skiprows=idx*batch_size, nrows=batch_size)
    df.columns = ['id', 'label']
    train_x, train_y = [], []
    for label in df['id']:
        train_x.append(np.asarray(Image.open(dirname+str(label)+".tif").resize((600, 1000))).reshape(600, 1000, 1)/255)
        temp = [0]*16
        temp[df.loc[df['id'] == label, 'label'].item()] = 1
        train_y.append(temp)
    train_x = np.array(train_x)
    train_y = np.array(train_y)
    return (train_x, train_y)

batch_size = 64
nb_epoch = 12
steps_per_epoch=np.ceil(16000/batch_size)

model = Sequential()
model.add(Conv2D(16, (5,5), padding="same", input_shape=(600, 1000, 1)))
model.add(MaxPooling2D())
model.add(Conv2D(32, (3,3), padding="same"))
model.add(MaxPooling2D())
model.add(Conv2D(64, (3,3), padding="same"))
model.add(MaxPooling2D())
model.add(Conv2D(128, (3,3), padding="same"))
model.add(MaxPooling2D())
model.add(Flatten())
model.add(Dense(100, activation='relu'))
model.add(Dense(16, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer="adam", metrics=["accuracy"])
model.summary()

model_checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath = './checkpt.h5',monitor='accuracy', mode='max', save_best_only=True)
callbacks_list = [model_checkpoint]
training_batch_generator = batch_generator('/kaggle/input/datathonindoml-2022/train_labels.csv', batch_size, steps_per_epoch)

Sat Nov 19 08:16:11 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.82.01    Driver Version: 470.82.01    CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    25W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

2022-11-19 08:16:11.754446: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-19 08:16:11.850841: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-19 08:16:11.851669: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-19 08:16:11.853705: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 600, 1000, 16)     416       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 300, 500, 16)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 300, 500, 32)      4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 150, 250, 32)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 150, 250, 64)      18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 75, 125, 64)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 75, 125, 128)      7

In [2]:
model.fit_generator(training_batch_generator, epochs=nb_epoch,steps_per_epoch=steps_per_epoch, callbacks=callbacks_list, verbose=1)

/opt/conda/lib/python3.7/site-packages/keras/engine/training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '
2022-11-19 08:16:15.932490: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/12


2022-11-19 08:16:17.755435: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


250/250 [==============================] - 315s 1s/step - loss: 3.4969 - accuracy: 0.1645
Epoch 2/12
250/250 [==============================] - 226s 905ms/step - loss: 2.0523 - accuracy: 0.3553
Epoch 3/12
250/250 [==============================] - 227s 911ms/step - loss: 1.5540 - accuracy: 0.5093
Epoch 4/12
250/250 [==============================] - 233s 932ms/step - loss: 1.1328 - accuracy: 0.6439
Epoch 5/12
250/250 [==============================] - 227s 912ms/step - loss: 0.6851 - accuracy: 0.7918
Epoch 6/12
250/250 [==============================] - 228s 915ms/step - loss: 0.3717 - accuracy: 0.8844
Epoch 7/12
250/250 [==============================] - 230s 923ms/step - loss: 0.1942 - accuracy: 0.9415
Epoch 8/12
250/250 [==============================] - 232s 929ms/step - loss: 0.1167 - accuracy: 0.9651
Epoch 9/12
250/250 [==============================] - 232s 931ms/step - loss: 0.0671 - accuracy: 0.9792
Epoch 10/12
250/250 [==============================] - 232s 930ms/step - loss:

In [3]:
from keras.models import load_model
model = load_model('./checkpt.h5')
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 600, 1000, 16)     416       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 300, 500, 16)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 300, 500, 32)      4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 150, 250, 32)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 150, 250, 64)      18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 75, 125, 64)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 75, 125, 128)      7

In [4]:
val_dir = '../input/datathonindoml-2022/validation/validation/'
results = []
for file in os.listdir(val_dir):
    image = np.asarray(Image.open(val_dir+file).resize((600, 1000))).reshape(1,600, 1000, 1)/255
    id_ = file.split(".")[0]
    label = np.argmax(model.predict(image) , axis=1)[0]
    results.append([id_, label])
df = pd.DataFrame(np.array(results), columns = ['id','label'])
df.to_csv('./results.csv')